# Iris Dataset Clustering

## Employ multiple clustering techniques on the Iris dataset, with and without PCA.  Evaluate results.

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.cluster import DBSCAN
import plotly.express as px

## Load Dataset, Explore and Display Features

In [ ]:
iris = load_iris()
iris_orig = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
iris_orig['target'] = iris_orig['target'].replace([0,1,2],['setosa', 'versicolor', 'virginica'])

* There are 3 unique target variables: setosa, versicolor and virginica; each a species of Iris
* Changed the target attribute labels to a descriptive string vs numerical category for ease in analysis

In [ ]:
iris_orig

In [ ]:
iris_orig.shape

In [ ]:
iris_orig.info()

* The dataset contains 150 observations, has 4 predictive attributes and 1 target variable
* The 4 predictive attributes are numerical, the target variable is categorical

In [ ]:
iris_orig['target'].describe()

In [ ]:
iris_orig.describe()

* petal length has the largest range and greatest variation of the 4 attributes, and also has the greatest difference of the 4 between its mean and median
* for the other three attributes, their mean approximates their median which suggests the mean is not affected by outliers

In [ ]:
profile = ProfileReport(iris_orig)
profile

In [ ]:
iris_orig.corr()

### Observations:

* The dataset has zero missing observations
* This is a balanced dataset in that each of the three target labels have the same number of observations
* The distributions of sepal length and sepal width are fairly normal
* The distributions of petal length and petal width both have two distinct groupings
* Correlation - because the 4 predictive attributes are all numerical, refer to the Pearson's r chart, above:
    * Sepal width and sepal length appear to be uncorrelated
    * Petal width and petal length appear to be highly correlated 
    * Petal length and sepal length appear to be fairly correlated
    * Petal width and sepal length also appear to be correlated, though less so than petal length and sepal length    
* Correlation - see pair plot graphs below for visual confirmation of the above correlation observations

## Build Elbow Plot to determine optimal number of clusters

In [ ]:
# Create Iris data frame without the target column

iris_features = iris_orig.drop(columns='target')
iris_features.head()

In [ ]:
# Create Elbow plot of inertia values to determine optimal number of clusters to use in a K-Means clustering method
# inertia is the sum of the squared distances of observations to their closest cluster center

inertia_values = []
cluster_centers = []
K = range(1,11) #Try number of clusters from 1 to 10

for k in K:
    k_mean_model = KMeans(n_clusters = k)
    k_mean_model.fit(iris_features)
    inertia_values.append(k_mean_model.inertia_) #track the inertia values for each number of clusters
    cluster_centers.append(k_mean_model.cluster_centers_) #track the cluster centers for each number of clusters

# Create data frame of values for elbow plot
elbow_data = {'Number of Clusters': K, 'Inertia': inertia_values}
elbow_df = pd.DataFrame(elbow_data) 

In [ ]:
# Graph the Elbow plot

fig_dims = (6, 3)
fig, ax = plt.subplots(figsize=fig_dims)

sns.set_theme(style = "whitegrid")
sns.pointplot(data = elbow_df, x = 'Number of Clusters' ,y = 'Inertia', markers=["o"])\
.set(title='Eblow Plot using Inertia for K-Means Clustering of Iris Data');

The optimal number of clusters is at the "elbow" of the graph, where the Inertia begins to decline in a linear fashion.  In this case the optimal number is 3.

In [ ]:
# Display cluster centers for k = 3 clusters in k-means model
cluster_centers[2:3]

## Mess with Scree plot - this is not on the Iris Data

In [ ]:
N=np.random.randn(6,9)
N=np.matrix(N.T)*np.matrix(N)
A,B,C=np.linalg.svd(N)
eigen_values=B**2/np.sum(B**2)
figure=plt.figure(figsize=(10,6))
sing_vals=np.arange(len(eigen_values)) + 1
plt.plot(sing_vals,eigen_values, 'ro-', linewidth=2)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue') 
plt.show() 

## Reduce data frame using Principal Components Analysis

In [ ]:
# PCA is affected by scale, so we need to scale our features before applying PCA
# StandardScaler will standardize the dataset’s features onto unit scale (mean = 0 and variance = 1) 

x = iris_features.values
scaled_array = StandardScaler().fit_transform(x) #This is an array of the scaled values of the four feature columns
iris_scaled = pd.DataFrame(data= np.c_[scaled_array], \
                             columns = ('sepal length', 'sepal width', 'petal length', 'petal width'))

# View scaled data frame
iris_scaled.head()

In [ ]:
# The first decision in PCA is to select the number of components to reduce to.
# The goal is to reduce dimensions while still retaining most of the variance of the features.

# Start with components=4 (use all) to assess what % of the variance each principal component contains
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(scaled_array)

print(pca.explained_variance_ratio_)
print(pca.explained_variance_ratio_.cumsum())

Note: based on the cumulative % of variance explained by the principal components, I can see that principal components 1, 2, and 3 contain 99.4% of the variation (information). 

In [ ]:
# Apply PCA to the scaled version of the Iris features, using 2 principal components (based on above analysis)
# Create and display PCA Features data frame. 
# Note: there isn't particular meaning assigned to each principal component, the new components are just the two main 
# dimensions of variation - which are linear combinations of the original variables.

iris_model = PCA(n_components=3)
principalComponents = iris_model.fit_transform(scaled_array)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
iris_PCA = pd.concat([principalDf, iris_orig[['target']]], axis = 1)
iris_PCA.head()

In [ ]:
# Scatterplot of the iris data reduced to 3 Principal Components

fig = px.scatter_3d(iris_PCA, x='principal component 1', y='principal component 2', z='principal component 3',
              color='target', size_max=8, opacity = .7)
fig.show()

## Employ Clustering Techniques - on original data frame and on PCA data frame
* Partitioning Method: K-Means
* Hierarchical Method: sklearn's AgglomerativeClustering
* Density-based Method: DBSCAN

### Partitioning Method: K-Means

#### K-Means on original Iris data set

In [ ]:
# Run KMeans on the original features of the Iris data frame, using 3 clusters

k_mean_model = KMeans(n_clusters = 3)
k_mean_model.fit(iris_features)

# Add columns to iris_features dataframe: predicted cluster from KMeans and target label from iris_df
iris_features['k_means']=k_mean_model.predict(iris_features)
iris_features['target']=iris_orig['target']

In [ ]:
# Display modified iris_features data frame
iris_features.head()

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=iris_features["k_means"],columns=iris_features["target"], margins=True) 
my_crosstab

The above table indicates an purity score of 89.3% ((36+50+48)/150 = .89333)

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index: ', metrics.adjusted_rand_score(iris_features["target"], iris_features["k_means"]),\
     ", measure the similarity of the two assignments: true label and predicted cluster")
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score: ',metrics.homogeneity_score(iris_features["target"], iris_features["k_means"]),\
     ", measures that each cluster contains only members of a single class")
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score: ', metrics.completeness_score(iris_features["target"], iris_features["k_means"]),\
     ", measures that all members of a given class are assigned to the same cluster")
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure: ', metrics.v_measure_score(iris_features["target"], iris_features["k_means"], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity")

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize = (15, 20))
sns.scatterplot(x='sepal length (cm)', y='sepal width (cm)', data = iris_features, hue='k_means',style='target',size='target', palette="deep",ax=ax1)\
.set(title='Sepal Length vs. Width: Color=K-Means, Shape=Actual Target')
sns.scatterplot(x='petal length (cm)', y='petal width (cm)', data = iris_features, hue='k_means',style='target',size='target',palette="deep",ax=ax2)\
.set(title='Petal Length vs. Width: Color=K-Means, Shape/Size=Actual Target');

#### K=Means on PCA Features data frame

In [ ]:
# Recall the structure of the PCA Features data frame

iris_PCA.head()

In [ ]:
# Run KMeans on the PCA Features data frame, using 3 clusters

k_mean_model = KMeans(n_clusters = 3)
k_mean_model.fit(iris_PCA[iris_PCA.columns[:-1]])

# Add column to iris_PCA_df dataframe: predicted cluster from KMeans
iris_PCA['k_means']=k_mean_model.predict(iris_PCA[iris_PCA.columns[:-1]])

In [ ]:
# Display modified iris_PCA_df data frame
iris_PCA.head()

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=iris_PCA_df["k_means"],columns=iris_PCA_df["target"], margins=True) 
my_crosstab

The above table indicates an purity score of 82.0% ((34+50+39)/150 = .82)

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index: ', metrics.adjusted_rand_score(iris_PCA_df["target"], iris_PCA_df["k_means"]),\
     ", measure the similarity of the two assignments: true label and predicted cluster")
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score: ',metrics.homogeneity_score(iris_PCA_df["target"], iris_PCA_df["k_means"]),\
     ", measures that each cluster contains only members of a single class")
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score: ', metrics.completeness_score(iris_PCA_df["target"], iris_PCA_df["k_means"]),\
     ", measures that all members of a given class are assigned to the same cluster")
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure: ', metrics.v_measure_score(iris_PCA_df["target"], iris_PCA_df["k_means"], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity")

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1)) = plt.subplots(1, figsize = (15, 8))
sns.scatterplot(x='principal component 1', y='principal component 2', data = iris_PCA_df, hue='k_means',style='target',size='target', palette="deep", ax=ax1)\
.set(title='Principal Component 1 vs. Principal Component 2: Color=K-Means, Shape/Size=Actual Target');

## Hierarchical Method: sklearn's AgglomerativeClustering

In [ ]:
iris_features.head()

In [ ]:
iris_features[iris_features.columns[:-2]].head()

In [ ]:
data_scaled = normalize(iris_features[iris_features.columns[:-2]])
data_scaled = pd.DataFrame(data_scaled, columns=iris_features.columns[:-2])
data_scaled.head()

In [ ]:
# Like an elbow plot for k-means, a dendogram helps determine the ideal number of clusters for hierarchical clustering
# Note: although this chart shows 2 clusters at the second level, we'll down to the 3rd level and use 3 clusters

plt.figure(figsize=(18, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(data_scaled, method='ward'))
plt.axhline(y=0.5, color='r', linestyle='--');

In [ ]:
# Note: the linkage criterion determines which distance to use between sets of observation.
# ‘ward’ minimizes the variance of the clusters being merged, euclidean is the distance used for this criterion

cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')  
cluster.fit_predict(data_scaled)

In [ ]:
data_scaled['hierarchical'] = cluster.fit_predict(data_scaled)
data_scaled['target'] = iris_features['target']
data_scaled.head()

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=data_scaled['hierarchical'],columns=data_scaled["target"], margins=True) 
my_crosstab

The above table indicates an purity score of 96.0% ((48+50+46)/150 = 0.96)

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index: ', metrics.adjusted_rand_score(data_scaled["target"], data_scaled['hierarchical']),\
     ", measure the similarity of the two assignments: true label and predicted cluster")
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score: ',metrics.homogeneity_score(data_scaled["target"], data_scaled['hierarchical']),\
     ", measures that each cluster contains only members of a single class")
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score: ', metrics.completeness_score(data_scaled["target"], data_scaled['hierarchical']),\
     ", measures that all members of a given class are assigned to the same cluster")
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure: ', metrics.v_measure_score(data_scaled["target"], data_scaled['hierarchical'], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity")

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize = (15, 15))
sns.scatterplot(x='sepal length (cm)', y='sepal width (cm)', data = data_scaled, hue='hierarchical',style='target',size='target', palette="deep",ax=ax1)\
.set(title='Sepal Length vs. Width: Color=hierarchical cluster, Shape/Size=Actual Target')
sns.scatterplot(x='petal length (cm)', y='petal width (cm)', data = data_scaled, hue='hierarchical',style='target',size='target',palette="deep",ax=ax2)\
.set(title='Petal Length vs. Width: Color=hierarchical cluster, Shape/Size=Actual Target');# Scatterplot of the iris data reduced to 3 Principal Components

# Creating 3D axes
# fig = plt.figure(figsize=(6,6))
# ax = Axes3D(fig, auto_add_to_figure=False)
# fig.add_axes(ax)

# cmap = ListedColormap(sns.color_palette("husl", 256).as_hex()) # Colormap magic

# scatter_3d = ax.scatter(pca_3_df['PC1'], pca_3_df['PC2'], pca_3_df['PC3'], c=colors,
#                          marker='o', cmap=cmap, alpha=1)

# ax.set_xlabel('PC1')
# ax.set_ylabel('PC2')
# ax.set_zlabel('PC3')
# legend = ax.legend(*scatter_3d.legend_elements(), title='Class')
# ax.add_artist(legend)
# plt.show()

fig2 = px.scatter_3d(x=iris_PCA['principal component 1'], y=iris_PCA['principal component 2'], 
                     z=iris_PCA['principal component 3'], color=iris_PCA['target'])
fig2.show()

### Density-based Method: DBSCAN

The scikit-learn implementation provides a default for the eps and min_samples parameters, but you’re generally expected to tune those. The eps parameter is the maximum distance between two data points to be considered in the same neighborhood. The min_samples parameter is the minimum amount of data points in a neighborhood to be considered a cluster.

In [ ]:
x = iris_features[iris_features.columns[:-2]]
x

In [ ]:
# Check through this example - need to standardize? normalize? Look up diff between the two and when they should be used.
# Do for original df and for pca separately
# eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
# min_samples: The number of samples (or total weight) in a neighborhood for a point to be considered as a core point
# noise: points that don't have the min # of points within the eps distance (not core points)
dbscan=DBSCAN(eps = 0.45, min_samples = 5)
dbscan.fit(x)
pca=PCA(n_components=2).fit(x)
pca_2d=pca.transform(x)

for i in range(0, pca_2d.shape[0]):
    if dbscan.labels_[i] == 0:
        c1 = plt.scatter(pca_2d[i, 0], pca_2d[i, 1], c='r', marker='+')
    elif dbscan.labels_[i] == 1:
        c2 = plt.scatter(pca_2d[i, 0], pca_2d[i, 1], c='g', marker='o')
    elif dbscan.labels_[i] == -1:
        c3 = plt.scatter(pca_2d[i, 0], pca_2d[i, 1], c='b', marker='*')
        
plt.legend([c1, c2, c3], ['Cluster 1', 'Cluster 2', 'Noise'])
plt.title('DBSCAN finds 2 clusters and Noise')
plt.show()

In [ ]:
# Check through this example - need to standardize? normalize? Look up diff between the two and when they should be used.
# Do for original df and for pca separately
# eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other
# min_samples: The number of samples (or total weight) in a neighborhood for a point to be considered as a core point
# noise: points that don't have the min # of points within the eps distance (not core points)
dbscan=DBSCAN(eps = 0.45, min_samples = 5)
dbscan.fit(x)

In [ ]:
# Add columns to iris_features dataframe: predicted cluster from KMeans and target label from iris_df
x['DBSCAN_cluster']=dbscan.labels_
x['target']=iris_orig['target']
x

In [ ]:
# Create table to assess clustering accuracy

my_crosstab = pd.crosstab(index=x['DBSCAN_cluster'],columns=x['target'], margins=True) 
my_crosstab

The above table indicates an purity score of 84.0% ((48+43+35)/150 = 0.84)- NOT SURE THIS CALC APPLIES HERE B/C it only produced two clusters.

In [ ]:
# Clustering performance evaluation - additional measures

# Rand Index - Adjusted: a function that measures the similarity of the two assignments, adjusted corrects for chance
print('Adjusted Rand Index: ', metrics.adjusted_rand_score(x["target"], x['DBSCAN_cluster']),\
     ", measure the similarity of the two assignments: true label and predicted cluster")
#homogeneity_score: each cluster contains only members of a single class.
print('Homogeneity_score: ',metrics.homogeneity_score(x["target"], x['DBSCAN_cluster']),\
     ", measures that each cluster contains only members of a single class")
#completeness_score: all members of a given class are assigned to the same cluster.
print('Completeness_score: ', metrics.completeness_score(x["target"], x['DBSCAN_cluster']),\
     ", measures that all members of a given class are assigned to the same cluster")
#v_measure_score: harmonic mean of homogeneity and completeness).
print('V-measure: ', metrics.v_measure_score(x["target"], x['DBSCAN_cluster'], beta=1.0),\
     ", harmonic mean of homogeneity and completeness, beta>1 gives more weight to homogeneity")

In [ ]:
# Graph actual labels vs. clustering for sepal length/sepal width and petal length/petal width

fig, ((ax1, ax2)) = plt.subplots(2, 1, figsize = (15, 20))
sns.scatterplot(x='sepal length (cm)', y='sepal width (cm)', data = x, hue='DBSCAN_cluster',style='target',size='target', palette="deep",ax=ax1)\
.set(title='Sepal Length vs. Width: Color=DBSCAN_cluster, Shape=Actual Target')
sns.scatterplot(x='petal length (cm)', y='petal width (cm)', data = x, hue='DBSCAN_cluster',style='target',size='target',palette="deep",ax=ax2)\
.set(title='Petal Length vs. Width: Color=DBSCAN_cluster, Shape/Size=Actual Target');

## Conclusions: 